# Generate hydrographs from digital elevation model (DEM)
This notebook demonstrates with a small examplary region (basin upstream of Altenahr, West Germany) how geomorphological unit hydrographs can be derived from a digital elevation model with a spatial resolution of 25m x 25m. Later on the hydrographs are the basis for the quick runoff (QR) modelling and allow for a large number of counterfacual simulations (see notebook XXXXXX.ipynb).

In [1]:
import counterfactual_functions as cf

In [2]:
input_path = "input/"

## Creation of basic rasters
Firstly, we need to transform the DEM into the PCRaster format (".map"). Follwing rasters are then created using PCRaster functions. The details can be seen in the script counterfactual functions.py

- sink filling is applied to the DEM ("dem_filled.map")
- flow direction grid ("flowdir.map")
- flow accumulation grid ("accu.map")

Outlets are determined by the intersection of streams with the Strahler-oder 7 upwards with the functions cf.create_outlet_raster and cf.find_intersects and stored in the raster "outlets.map".

Then outlet and flow direction raster are used to delineate the subbasins. Very small subbasins (< 0.45 km²) are removed with the function cf.remove_small_subbasins.





In [3]:
cf.create_basic_files(input_path)

Input file size is 1591, 1453
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating filled DEM
Creating flow direction grid
Creating accumulation grid
Creating subbasin shape file and raster


/home/voit/GIUH/paper_repo/counterfactual_flash_flood_analysis/counterfactual_functions.py:385: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '9.25875' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  size_df.loc[size_df.id == id, "size_km2"] = (size * 25 * 25) / 1e6


Removed 19 outlets
The basin now contains 57 subbasins
0...10...20...30...40...50...60...70...80...90...Creating output input/subbasins_first.gpkg of format GPKG.
100 - done.
Making maidment friction map
Number of outlets: 58
2023-12-18 14:18:18.990012 Starting traveltime
2023-12-18 14:18:19.698294 Traveltime complete
2023-12-18 14:18:19.698974Creating soil landuse shape file for curve number method


FileNotFoundError: [Errno 2] No such file or directory: 'input/scs.csv'

In [ ]:
import pcraster as pcr
test = pcr.readmap("/home/voit/GIUH/main_basins/ger/input/ger_accu.map")